In [3]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import PorterStemmer
# from nltk.corpus import stopwords
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import WordNetLemmatizer 
import spacy

In [ ]:
# stop_words_list = stopwords.words('english')
with open("stop_words") as file:
    stop_words_list = [line.strip() for line in file]


# Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
spacy_lemmatizer = spacy.load('en', disable=['parser', 'ner'])



## process Tweets

In [176]:
file_name = "amazon.csv"
sample_size = 30000
data = pd.read_csv('data_files/twitter/'+file_name)

data['status_id'] = data['status_id'].str.strip()
data.drop_duplicates(subset='status_id', keep = 'last', inplace = True)
data.shape

(35610, 88)

In [177]:
dt = data.sample(n = sample_size)
dt.shape

(30000, 88)

In [178]:
with open('processed_data/twitter/' + file_name, 'w') as f:
    for index, row in dt.iterrows():
            rowdata = row["text"]
            file_clear = re.sub("(@[A-Za-z0-9]+)|([^A-Za-z \t])|(\w+:\/\/\S+)", " ", rowdata.lower())
            file_lem = spacy_lemmatizer(file_clear)
            file_lem = " ".join([token.lemma_ for token in file_lem])
            tokens = nltk.word_tokenize(file_lem)
            filtered_words = [word for word in tokens if word not in stop_words_list]
            for item in filtered_words:
                f.write("%s " % item)
f.close()

## Process Nytimes data

In [133]:
file_name = 'nyt_apple.csv'
sample_size = 300
article_df = pd.read_csv('data_files/nyt/' + file_name)

In [134]:
article_df['id'] = article_df['id'].str.strip()
article_df.drop_duplicates(subset='id', keep = 'last', inplace = True)
article_df = article_df.dropna()
# article_df.sort_values("id", inplace = True) 
article_df.shape

(457, 3)

In [135]:
dt = article_df.sample(n = sample_size)
dt.shape

(300, 3)

In [137]:
with open('processed_data/nyt/' + file_name, 'w') as f:
    for index, row in dt.iterrows():
            rowdata = row["content"]
            file_clear = re.sub("(@[A-Za-z0-9]+)|([^A-Za-z \t])|(\w+:\/\/\S+)", " ", rowdata.lower())
            file_lem = spacy_lemmatizer(file_clear)
            file_lem = " ".join([token.lemma_ for token in file_lem])
            tokens = nltk.word_tokenize(file_lem)
            filtered_words = [word for word in tokens if word not in stop_words_list]
            for item in filtered_words:
                f.write("%s " % item)
f.close()



## Sort and get data

In [55]:
file_name = "uber"
# data = pd.read_csv('mr_output/nyt/'+file_name, sep='\t',header=None, names=['word','count'])
data = pd.read_csv('mr_output/twitter/'+file_name, sep='\t',header=None, names=['word','count'])

In [56]:
data.sort_values(by=['count'], ascending=False).head(50)

,word,count
19066,uber,27449
5500,driver,6721
10991,lyft,4129
9450,ipo,2780
15383,ride,2461
5702,eat,1975
5496,drive,1870
2777,car,1808
18487,time,1619
1812,billion,1594
